# Balancierung und Erweiterung der Klassifikations Datensätze


## Einstellungen & Import

In [ ]:
import pandas as pd
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
import numpy as np
import smogn
import os

from scipy.stats import zscore
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# Set style for plots
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 12
#%config InlineBackend.figure_format = 'svg'

# Test Split 
test_size = 0.2

In [ ]:
# Lade den DataFrame aus der Datei
df = pd.read_pickle("../datasets/new_features.pkl")

# Überprüfe die Struktur des geladenen DataFrames
print("Shape des geladenen DataFrames:", df.shape)
df.head()

## Hilfsfunktionen

In [ ]:
def stratified_split(df, target_column, test_size=0.2, random_state=42):
    """
    Führt einen stratifizierten Split basierend auf der Zielspalte durch.

    Args:
        df (pd.DataFrame): Der Eingabe-DataFrame.
        target_column (str): Die Zielspalte für den Stratified Split.
        test_size (float): Anteil der Testdaten.
        random_state (int): Seed für die Reproduzierbarkeit.

    Returns:
        tuple: Trainings- und Testdatensätze.
    """
    strat_split = StratifiedShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    for train_idx, test_idx in strat_split.split(df, df[target_column]):
        train_df = df.iloc[train_idx].reset_index(drop=True)
        test_df = df.iloc[test_idx].reset_index(drop=True)
    return train_df, test_df

In [ ]:
def balance_and_augment_with_smote(train_df, target_column, target_count_per_class):
    """
    Balanciert den Trainingsdatensatz durch Hinzufügen synthetischer Instanzen
    mithilfe von SMOTE, ohne Originaldaten zu reduzieren, und erreicht eine feste Zielgröße pro Klasse.

    Args:
        train_df (pd.DataFrame): Trainingsdatensatz.
        target_column (str): Die Zielspalte, die balanciert werden soll.
        target_count_per_class (int): Zielgröße für jede Klasse.

    Returns:
        pd.DataFrame: Der ausgeglichene Trainingsdatensatz mit allen Originaldaten und zusätzlichen synthetischen Daten.
    """
    smote = SMOTE(sampling_strategy='auto', random_state=42)
    
    # Trennen der Features und der Zielspalte
    X = train_df.drop(columns=[target_column, 'VersuchID'], errors='ignore')  # 'VersuchID' nicht für SMOTE verwenden
    y = train_df[target_column]

    # Speichere die Originaldaten mit Label
    original_data = train_df.copy()
    original_data['synthetisch'] = False  # Markiere Originaldaten als nicht synthetisch

    # SMOTE anwenden
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Kombinieren der Originaldaten und der SMOTE-Daten
    smote_generated_df = pd.DataFrame(X_resampled, columns=X.columns)
    smote_generated_df[target_column] = y_resampled

    # **Fix für korrektes Labeling: Synthetische Daten markieren, aber Originale beibehalten**
    smote_generated_df['synthetisch'] = True
    smote_generated_df.loc[smote_generated_df.index < len(original_data), 'synthetisch'] = False

    # Zielgröße pro Klasse sicherstellen
    final_df = [smote_generated_df]  # Füge die vollständige SMOTE-Datenmenge ein
    for cls in smote_generated_df[target_column].unique():
        cls_data = smote_generated_df[smote_generated_df[target_column] == cls]
        if len(cls_data) < target_count_per_class:
            # Zusätzliche Instanzen mit SMOTE generieren
            additional_data = cls_data.sample(
                n=target_count_per_class - len(cls_data),
                replace=True,
                random_state=42
            )
            additional_data['synthetisch'] = True  # Markiere zusätzliche Daten als synthetisch
            final_df.append(additional_data)

    return pd.concat(final_df, ignore_index=True)

def process_targets_with_smote(df, target_columns, test_size=0.2, target_count_per_class=500):
    """
    Splittet die Daten und balanciert jede Zielspalte mithilfe von SMOTE,
    ohne Originaldaten zu entfernen.

    Args:
        df (pd.DataFrame): Der Eingabe-DataFrame.
        target_columns (list): Liste der Zielspalten.
        test_size (float): Anteil der Testdaten.
        target_count_per_class (int): Anzahl der Instanzen pro Klasse nach Balancing.

    Returns:
        dict: Trainings- und Testdatensätze für jede Zielspalte.
    """
    datasets = {}

    for target in target_columns:
        print(f"\n>>> Verarbeitung für Zielgröße: {target} <<<")
        
        # Stratified Split
        train_df, test_df = stratified_split(df, target, test_size=test_size)

        print(f"Verteilung vor Balancing im Trainingsdatensatz für '{target}':")
        print(train_df[target].value_counts())

        # Balancing mit SMOTE ohne Entfernen von Originaldaten
        train_balanced_df = balance_and_augment_with_smote(train_df, target, target_count_per_class)

        print(f"Verteilung nach Balancing im Trainingsdatensatz für '{target}':")
        print(train_balanced_df[target].value_counts())

        # Speichern der Ergebnisse
        datasets[target] = {
            "train": train_balanced_df,
            "test": test_df
        }

    return datasets

## SMOTE für Klassifikations-Zielgrößen

In [ ]:
# Zielspalten (Anpassen an Ihren Datensatz)
target_columns = ['Ergebnis_con', 'Material_con', 'Position_con', 'richtig_verbaut']
target_count = 2500
# Verarbeitung der Zielgrößen
if test_size == 0.3:
        datasets = process_targets_with_smote(df, target_columns, test_size=0.3, target_count_per_class=target_count)
else:
    datasets = process_targets_with_smote(df, target_columns, test_size=0.2, target_count_per_class=target_count)

In [ ]:
# # Speichern der Ergebnisse
# for target, data in datasets.items():
    
#     if test_size == 0.3:
#         train_path = f"../datasets/train_balanced_{target}_smote_testsize03_{target_count}.pkl"
#         test_path = f"../datasets/test_{target}_30.pkl"
#     else:
#         train_path = f"../datasets/train_balanced_{target}_smote_{target_count}.pkl"
#         test_path = f"../datasets/test_{target}.pkl"
    
    
#     data["train"].to_pickle(train_path)
#     data["test"].to_pickle(test_path)

#     print(f"Trainings- und Testdatensätze für '{target}' gespeichert.")

### Visualisierung

In [ ]:
# Dictionary der Datensätze für Zielgrößen
datasets_classification = {
    "Ergebnis_con": {
        "train": f"../Datasets/train_balanced_Ergebnis_con_smote_{target_count}.pkl",
        "test": "../Datasets/test_Ergebnis_con.pkl"
    },
    "Material_con": {
        "train": f"../Datasets/train_balanced_Material_con_smote_{target_count}.pkl",
        "test": "../Datasets/test_Material_con.pkl"
    },
    "Position_con": {
        "train": f"../Datasets/train_balanced_Position_con_smote_{target_count}.pkl",
        "test": "../Datasets/test_Position_con.pkl"
    },
    "richtig_verbaut": {
        "train": f"../Datasets/train_balanced_richtig_verbaut_smote_{target_count}.pkl",
        "test": "../Datasets/test_richtig_verbaut.pkl"
    },
}

In [ ]:
def create_pairplot(data, target_column, title, ignored_features=None):
    """
    Erstellt einen Pairplot für einen gegebenen DataFrame.

    Args:
        data (pd.DataFrame): Der Eingabedatensatz.
        target_column (str): Die Zielspalte.
        title (str): Der Titel des Plots.
        ignored_features (list, optional): Liste der zu ignorierenden Features. Standard ist None.

    Returns:
        None
    """
    print(f"Datensatzgröße: {data.shape}")
    print(data[target_column].value_counts())

    # Ignoriere ausgewählte Features
    if ignored_features:
        data = data.drop(columns=ignored_features, errors='ignore')
    
    # Wähle numerische Spalten und Zielspalte
    numeric_cols = data.select_dtypes(include=['float64', 'int']).columns
    numeric_cols = [col for col in numeric_cols if col != target_column]
    
    if len(numeric_cols) > 5:  # Maximal 5 Spalten für Übersichtlichkeit
        numeric_cols = numeric_cols[:5]
    
    # Erstelle den Pairplot und unterscheide Klassen durch Farben
    g = sns.pairplot(
        data[numeric_cols + [target_column]],
        hue=target_column,
        diag_kind="kde",
        height=2.5,  # Erhöht die Größe der Plots
        plot_kws={'alpha': 0.6, 's': 20},  # Transparenz und Punktgröße anpassen
        markers=["o", "s"],  # Markierung für die Klassen
    )
    g.fig.suptitle(title, y=1.02)
    plt.show()

In [ ]:
# Liste der zu ignorierenden Features
ignored_features = []  # Beispiel: ['Feature1', 'Feature2']

# Zielspalten für jede Zielgröße
target_columns = {
    "Ergebnis_con": "Ergebnis_con",
    "Material_con": "Material_con",
    "Position_con": "Position_con",
    "richtig_verbaut": "richtig_verbaut",
    #"Probenhoehe": "Probenhoehe"
}

# Generiere Pairplots und Klassenverteilungen für die Trainingsdatensätze
for target, paths in datasets_classification.items():
    print(f"Pairplot für Trainingsdatensatz: {target}")
    # Laden des DataFrames
    train_data = pd.read_pickle(paths["train"])
    create_pairplot(train_data, target_columns[target], f"Train: {target}", ignored_features)

#### Real vs. Synthetisch

In [ ]:
# Datei laden
zielgroeße = "Position_con"
df_train = pd.read_pickle(f"../datasets/train_balanced_{zielgroeße}_smote_2500.pkl")


In [ ]:
print(df_train.groupby(['synthetisch', zielgroeße]).size())

In [ ]:
# Annahme: Die Features sind alle Spalten außer der Zielvariable & kategorischen Spalten
features = [col for col in df_train.columns if col not in ['Ergebnis_con', 'VersuchID', 'Material_con', 'Position_con', 'richtig_verbaut','Probenhoehe']]


# # Boxplots getrennt nach Ergebnis_con (0 vs. 1)
# for feature in features:
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='synthetisch', y=feature, hue='Ergebnis_con', data=df_train, palette="Set2")
    
#     plt.title(f'Vergleich der Verteilung für {feature} nach Klassen')
#     plt.xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
#     plt.ylabel(feature)
#     plt.legend(title="Ergebnis_con", loc="upper right")
    
#     plt.show()

# Setze die Features für die Subplots
features_to_plot = ["Berührzeit", "Motorstrom_Durchschnitt"]

# Erstelle die Subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))  # 1 Zeile, 2 Spalten

for i, feature in enumerate(features_to_plot):
    sns.boxplot(x='synthetisch', y=feature, hue=zielgroeße, data=df_train, palette="Set2", ax=axes[i])
    axes[i].set_title(f'Vergleich der Verteilung für {feature} nach Klassen')
    axes[i].set_xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
    axes[i].set_ylabel(feature)
    axes[i].legend(title=zielgroeße, loc="upper right")

# Subplots platzsparend anordnen
plt.tight_layout()
plt.show()

# Erweiterung Probenhöhe


In [ ]:
# **Spezifische Anwendung für Probenhöhe**
target_column = "Probenhoehe"

# Datensatz laden
df = pd.read_pickle("../datasets/new_features.pkl")

## Hilfsfunktionen

In [ ]:
def detect_and_visualize_outliers_by_group(df, column_name, group_col):
    """
    Identifiziert und visualisiert Outlier in 'column_name' getrennt nach 'group_col'.
    
    Args:
        df (pd.DataFrame): DataFrame mit den Daten.
        column_name (str): Name der Spalte, in der Outlier identifiziert werden sollen.
        group_col (str): Name der Spalte, nach der gruppiert werden soll (z.B. 'NominalProbenhoehe').

    Returns:
        pd.DataFrame: DataFrame mit allen IQR-Outliern aus allen Gruppen.
        pd.DataFrame: DataFrame mit allen Z-Score-Outliern aus allen Gruppen.
    """
    # Listen zum Sammeln aller Outlier pro Gruppe
    outliers_iqr_list = []
    outliers_zscore_list = []

    # Gruppieren nach der gewünschten Spalte (z.B. nominale Probenhöhe)
    grouped = df.groupby(group_col)

    for group_value, group_df in grouped:
        print(f"\n--- Auswertung für {group_col} = {group_value} ---")

        # 1) IQR-Methode
        Q1 = group_df[column_name].quantile(0.25)
        Q3 = group_df[column_name].quantile(0.75)
        IQR = Q3 - Q1
        
        # Grenzen anpassen, falls gewünscht (hier 1.2*IQR wie in deinem Code)
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_iqr = group_df[
            (group_df[column_name] < lower_bound) | (group_df[column_name] > upper_bound)
        ]
        outliers_iqr_list.append(outliers_iqr)

        print(f"  Anzahl Outlier (IQR) für Gruppe {group_value}: {len(outliers_iqr)}")

        # 2) Z-Score-Methode
        # Wir berechnen den Z-Score nur auf die Teilgruppe
        group_df = group_df.copy()  # damit wir den Original-DataFrame nicht überschreiben
        group_df['Z-Score'] = zscore(group_df[column_name])
        outliers_z = group_df[np.abs(group_df['Z-Score']) > 3]
        outliers_zscore_list.append(outliers_z)
        
        print(f"  Anzahl Outlier (Z-Score) für Gruppe {group_value}: {len(outliers_z)}")

        # -- Visualisierung pro Gruppe (optional) --
        # Boxplot
        plt.figure(figsize=(10, 4))
        sns.boxplot(x=group_df[column_name])
        plt.title(f"Boxplot der {column_name} (Gruppe: {group_value})")
        plt.show()

        # Histogramm
        plt.figure(figsize=(10, 4))
        sns.histplot(group_df[column_name], bins=30, kde=True, color="blue")
        plt.axvline(lower_bound, color="red", linestyle="--", label="IQR Untergrenze")
        plt.axvline(upper_bound, color="red", linestyle="--", label="IQR Obergrenze")
        plt.legend()
        plt.title(f"Histogramm der {column_name} (Gruppe: {group_value})")
        plt.show()

    # Zusammenführen aller Outlier aus allen Gruppen
    outliers_iqr_all = pd.concat(outliers_iqr_list) if outliers_iqr_list else pd.DataFrame()
    outliers_zscore_all = pd.concat(outliers_zscore_list) if outliers_zscore_list else pd.DataFrame()

    return outliers_iqr_all, outliers_zscore_all

def remove_outliers_by_group(df, column_name, group_col):
    """
    Entfernt Outlier aus dem DataFrame, gruppiert nach einer nominalen Spalte.
    Die Outlier werden mithilfe der IQR-Methode innerhalb jeder Gruppe identifiziert.
    Anschließend wird der DataFrame bereinigt und die Gruppierungsspalte entfernt.

    Args:
        df (pd.DataFrame): Ursprünglicher DataFrame.
        column_name (str): Name der Spalte, in der Outlier erkannt werden sollen.
        group_col (str): Name der Spalte, die die nominalen Werte (z.B. 38, 40, 42) enthält.

    Returns:
        pd.DataFrame: Bereinigter DataFrame ohne Outlier und ohne die Gruppierungsspalte.
    """
    # Kopie erstellen, um das Original beizubehalten
    df_clean = df.copy()
    outlier_indices = []

    # Gruppierung nach der nominalen Probenhöhe (die zuvor aus den IST-Werten erstellt wurde)
    for group_value, group_df in df_clean.groupby(group_col):
        Q1 = group_df[column_name].quantile(0.25)
        Q3 = group_df[column_name].quantile(0.75)
        IQR = Q3 - Q1
        
        # Grenzen der IQR-Methode (hier 1.2 * IQR, kann angepasst werden)
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identifikation der Outlier in dieser Gruppe
        group_outliers = group_df[(group_df[column_name] < lower_bound) | (group_df[column_name] > upper_bound)]
        outlier_indices.extend(group_outliers.index.tolist())

    # Doppelte Indizes entfernen
    outlier_indices = list(set(outlier_indices))
    
    # Entferne die Zeilen mit Outliern aus dem DataFrame
    df_clean = df_clean.drop(index=outlier_indices)
    
    # Entferne die nominale Spalte, die zur Gruppierung genutzt wurde
    df_clean = df_clean.drop(columns=[group_col])
    
    return df_clean



In [ ]:
# Definiere die nominalen Werte
nominal_values = np.array([38, 40, 42])

# Funktion, um den nächstgelegenen nominalen Wert zu bestimmen
def assign_nominal_value(ist_value, nominal_values=nominal_values):
    distances = np.abs(nominal_values - ist_value)
    return nominal_values[np.argmin(distances)]

# Hilfssplte für Darstellung
df['NominalProbenhoehe'] = df['Probenhoehe'].apply(assign_nominal_value)

In [ ]:
print("Originale Daten:")
outlier_df1 = detect_and_visualize_outliers_by_group(df, "Probenhoehe",group_col="NominalProbenhoehe")

In [ ]:
# DataFrame ohne Outlier
df_clean = remove_outliers_by_group(df, column_name="Probenhoehe", group_col="NominalProbenhoehe")

In [ ]:
# Visualisierung bereinigter original Daten

plt.figure(figsize=(10, 5))
sns.histplot(df_clean['Probenhoehe'], bins=300, kde=True, color="#4C72B0")
plt.title("Verteilung der Probenhöhe im bereinigten originalen Datensatz")
plt.xlabel("Probenhöhe (mm)")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Split in Training und Test
train_df, test_df = train_test_split(df_clean, test_size=test_size, random_state=42)
train_df.to_pickle(f"../datasets/train_{target_column}_clean.pkl")
test_df.to_pickle(f"../datasets/test_{target_column}_clean.pkl")

## SMOGN

### Hilfsfunktionen

In [ ]:
def apply_smogn_with_inverse_scaling(df, target_col, mode_ranges, samples_per_mode=500, k=5, rel_thres=0.8, random_state=42, use_manual_relevance=False):
    """
    Erzeugt synthetische Daten mit SMOGN und führt anschließend eine inverse Skalierung und Rücktransformation der label-encodeten Spalten durch.
    
    Parameter:
      df: Ursprünglicher DataFrame.
      target_col: Name der Zielspalte.
      mode_ranges: Liste von Tupeln, die die Bereiche definieren (min, max) für die SMOGN-Anwendung.
      samples_per_mode: Maximale Anzahl synthetischer Samples pro Bereich.
      k: Parameter für SMOGN (Anzahl der Nachbarn).
      rel_thres: Schwellenwert für Relevanz.
      random_state: Zufallsseed.
      use_manual_relevance: Falls True, wird die manuelle Relevanzfunktion verwendet (mit definierten Kontrollpunkten),
                              andernfalls wird SMOGNs automatischer Relevanzmechanismus genutzt.
    """
    df = df.copy()
    np.random.seed(random_state)

    # --- Kategorische Spalten erkennen und label-encoden ---
    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    label_encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    # --- Fehlende Werte behandeln ---
    imputer = SimpleImputer(strategy='median')
    df[df.columns] = imputer.fit_transform(df)

    # --- Skalierung (nur Features) ---
    scaler = StandardScaler()
    X = df.drop(columns=[target_col])
    y = df[target_col]
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    df_scaled = pd.concat([X_scaled, y.reset_index(drop=True)], axis=1)

    # --- Relevanzparameter definieren ---
    if use_manual_relevance:
        rel_method = "manual"
        rel_ctrl_pts_rg = [
            [35.0, 0.0],
            [38.0, 1.0],
            [38.5, 0.0],
            [39.5, 0.0],
            [40.0, 1.0],
            [40.5, 0.0],
            [41.5, 0.0],
            [42.0, 1.0],
            [42.5, 0.0],
            [45.0, 0.0]
        ]
    else:
        rel_method = "auto"
        rel_ctrl_pts_rg = None  # wird bei "auto" nicht benötigt

    all_augmented = []

    for min_val, max_val in mode_ranges:
        try:
            # Auswahl des Teilbereichs basierend auf dem Zielwert
            subset = df_scaled[(df_scaled[target_col] >= min_val) & (df_scaled[target_col] <= max_val)].copy()
            if subset.empty or subset.shape[0] < k + 1:
                print(f"Zu wenige Daten im Bereich {min_val}-{max_val}, überspringe...")
                continue

            subset.reset_index(drop=True, inplace=True)
            original_columns = subset.columns.tolist()
            print(f"[DEBUG] Bereich {min_val}-{max_val}: subset.shape = {subset.shape}")

            # --- SMOGN-Anwendung ---
            if use_manual_relevance:
                smogn_result = smogn.smoter(
                    data=subset.copy(),
                    y=target_col,
                    samp_method="balance",
                    rel_method=rel_method,
                    rel_ctrl_pts_rg=rel_ctrl_pts_rg,
                    rel_thres=rel_thres,
                    k=k
                )
            else:
                smogn_result = smogn.smoter(
                    data=subset.copy(),
                    y=target_col,
                    samp_method="balance",
                    rel_method=rel_method,
                    rel_thres=rel_thres,
                    k=k
                )

            print(f"[DEBUG] SMOGN-Ergebnis vor Spaltenanpassung: shape = {smogn_result.shape}, columns = {list(smogn_result.columns)}")

            # Entferne evtl. vorhandene zusätzliche Spalte(n), z. B. "rel"
            extra_cols = [col for col in smogn_result.columns if col not in original_columns]
            if extra_cols:
                smogn_result.drop(columns=extra_cols, inplace=True)
                print(f"[DEBUG] Entfernte Spalten: {extra_cols}")

            # Prüfe, ob die Spaltenanzahl übereinstimmt
            if smogn_result.shape[1] != subset.shape[1]:
                print(f"Spaltenanzahl stimmt nicht überein nach SMOGN für Bereich {min_val}-{max_val}. Erhalten: {smogn_result.shape[1]}, erwartet: {subset.shape[1]}")
                continue

            # --- Extraktion der synthetisch generierten Zeilen ---
            new_rows = smogn_result.iloc[subset.shape[0]:].copy()
            print(f"[DEBUG] Für Bereich {min_val}-{max_val}: new_rows.shape = {new_rows.shape}")
            if new_rows.empty:
                print(f"Keine neuen Daten erzeugt für Bereich {min_val}-{max_val}")
                continue

            if new_rows.shape[0] > samples_per_mode:
                new_rows = new_rows.sample(n=samples_per_mode, random_state=random_state)

            new_rows["synthetisch"] = 1
            all_augmented.append(new_rows)

        except Exception as e:
            print(f"Fehler bei SMOGN für Bereich {min_val}-{max_val}: {e}")
            continue

    if not all_augmented:
        raise ValueError("Keine synthetischen Daten erzeugt.")

    # --- Rücktransformation der skalierten Features ---
    df_augmented_scaled = pd.concat(all_augmented, ignore_index=True)
    X_new = df_augmented_scaled.drop(columns=[target_col, "synthetisch"])
    X_new_inverse = pd.DataFrame(scaler.inverse_transform(X_new), columns=X.columns)
    df_augmented = pd.concat([
        X_new_inverse,
        df_augmented_scaled[[target_col, "synthetisch"]].reset_index(drop=True)
    ], axis=1)

    # --- Rücktransformation der label-encodeten kategorialen Variablen ---
    for col in cat_cols:
        le = label_encoders[col]
        df_augmented[col] = df_augmented[col].round().astype(int).clip(0, len(le.classes_) - 1)
        df_augmented[col] = le.inverse_transform(df_augmented[col])

    # --- Originaldaten (synthetisch = 0) hinzufügen ---
    df_original = df.copy()
    df_original["synthetisch"] = 0

    df_final = pd.concat([df_original, df_augmented], ignore_index=True)
    return df_final

def compute_phi_relevance(y, rel_thres=0.8):
    """
    Erzeugt Relevanzwerte wie in SMOGN bei rel_method="auto".
    """
    y = np.array(y)
    q1 = np.percentile(y, 25)
    q3 = np.percentile(y, 75)
    iqr = q3 - q1

    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr

    rel = np.where((y < lower_bound) | (y > upper_bound), 1, 0)
    return rel

In [ ]:
irrelevant_columns = ['Material_con', 'Position_con', 'Ergebnis_con', 'richtig_verbaut', 'Zeit', 'VersuchID','umformzeit' # Umformzeit wurde doppelt berechnet
                      ]
df_reduced = df_clean.drop(columns=irrelevant_columns, errors='ignore')

mode_ranges = [(37.5, 38.5), (39, 41), (41.5, 42.5)]  # Bereich in dem die originalen Daten liegen
df_augmented = apply_smogn_with_inverse_scaling(
    df_reduced,
    target_col="Probenhoehe",
    mode_ranges=mode_ranges,
    samples_per_mode=800
)

### Visualisierung

In [ ]:
# Visualisierung synthetischer Daten

plt.figure(figsize=(10, 5))
sns.histplot(df_augmented['Probenhoehe'], bins=300, kde=True, color="#4C72B0")
plt.title("Verteilung der Probenhöhe im augmentierten Datensatz")
plt.xlabel("Probenhöhe (mm)")
plt.ylabel("Häufigkeit")
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Hilfssplte für Visualisierung 
df_augmented['NominalProbenhoehe'] = df_augmented['Probenhoehe'].apply(assign_nominal_value)

outlier_df_test_jittered = detect_and_visualize_outliers_by_group(df_augmented, "Probenhoehe",group_col="NominalProbenhoehe")

In [ ]:
# Erneutes Entfernen der Outlier, falls erforderlich
clean_train_df = remove_outliers_by_group(df_augmented, column_name="Probenhoehe", group_col="NominalProbenhoehe")

In [ ]:
# Visualisierung Verteilung der Daten: original vs synthetisch

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(df_clean['Probenhoehe'], bins=300, kde=True, color="#4C72B0", ax=axs[0])
axs[0].set_title("Verteilung der Probenhöhe im bereinigten originalen Datensatz")
axs[0].set_xlabel("Probenhöhe (mm)")
axs[0].set_ylabel("Häufigkeit")
axs[0].grid(True)

sns.histplot(df_augmented['Probenhoehe'], bins=300, kde=True, color="#4C72B0", ax=axs[1])
axs[1].set_title("Verteilung der Probenhöhe im augmentierten Datensatz")
axs[1].set_xlabel("Probenhöhe (mm)")
axs[1].set_ylabel("Häufigkeit")
axs[1].grid(True)

plt.tight_layout()
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("ProbenhöheVerteilungSMOGN.svg", format="svg", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Hilfssplte für Visualisierung 
clean_train_df['NominalProbenhoehe'] = clean_train_df['Probenhoehe'].apply(assign_nominal_value)

outlier_df_test_jittered = detect_and_visualize_outliers_by_group(clean_train_df, "Probenhoehe",group_col="NominalProbenhoehe")

In [ ]:
# Entfernen NominalProbenhoehe
clean_train_df = clean_train_df.drop(columns=['NominalProbenhoehe',], errors='ignore')

In [ ]:
clean_train_df


In [ ]:
# Annahme: Die Features sind alle Spalten außer der Zielvariable & kategorischen Spalten
features = [col for col in clean_train_df.columns if col not in ['Ergebnis_con', 'VersuchID', 'Material_con', 'Position_con', 'richtig_verbaut','Probenhoehe']]


# Boxplots getrennt nach Ergebnis_con (0 vs. 1)
# for feature in features:
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='synthetisch', y=feature, data=clean_train_df, palette="Set2")
    
#     plt.title(f'Vergleich der Verteilung für {feature} nach Klassen')
#     plt.xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
#     plt.ylabel(feature)
#     plt.legend(title="Probenhoehe", loc="upper right")
    
#     plt.show()

# Setze die Features für die Subplots
features_to_plot = ["Berührzeit", "Verkippung_2_Min"]

# Erstelle die Subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))  # 1 Zeile, 2 Spalten

for i, feature in enumerate(features_to_plot):
    sns.boxplot(x='synthetisch', y=feature, data=clean_train_df, palette="Set2", ax=axes[i])
    axes[i].set_title(f'Vergleich der Verteilung für {feature} nach Klassen')
    axes[i].set_xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
    axes[i].set_ylabel(feature)

# Subplots platzsparend anordnen
plt.tight_layout()
plt.show()

In [ ]:
clean_train_df

In [ ]:
# Entfernen der Hilfsspalten 
df_train = clean_train_df.drop(columns=['synthetisch','Probenhoehe_Gruppe'], errors='ignore')

In [ ]:
# Speichern der Trainingsdaten
#df_train.to_pickle(f"../datasets/train_{target_column}_smogn.pkl")

## Jitter

### Hilfsfunktionen

In [ ]:
def extend_regression_target_with_jitter(df, target_column, new_sample_count, jitter_std=0.01):
    """
    Erweiterung der Trainingsdaten für ein Regressionsziel durch Jittering.
    
    Args:
        df (pd.DataFrame): Eingabedatensatz.
        target_column (str): Die Zielspalte (Regression).
        new_sample_count (int): Anzahl der zu generierenden neuen Samples.
        jitter_std (float): Standardabweichung für das Hinzufügen von Jitter.
        
    Returns:
        pd.DataFrame: Erweiterter Datensatz.
    """
    print(f"Originale Datenpunkte: {len(df)}")
    
    # Erzeuge neue Samples durch Jittering
    jittered_samples = []
    for _ in range(new_sample_count):
        # Zufällige Auswahl eines existierenden Samples
        sample = df.sample(n=1, random_state=np.random.randint(1000))
        jittered_sample = sample.copy()
        
        # Füge Jitter (zufällige Störungen) zu allen numerischen Features hinzu
        for col in df.columns:
            if col != target_column and np.issubdtype(df[col].dtype, np.number):
                jittered_sample[col] += np.random.normal(loc=0.0, scale=jitter_std)
        
        jittered_samples.append(jittered_sample)
    
    # Kombiniere Original- und Jittered-Daten
    extended_df = pd.concat([df] + jittered_samples, ignore_index=False)
    print(f"Erweiterte Datenpunkte: {len(extended_df)}")
    return extended_df

def detect_and_visualize_outliers(df, column_name):
    """
    Identifiziert und visualisiert Outlier in einer bestimmten Spalte eines DataFrames.

    Args:
        df (pd.DataFrame): DataFrame mit den Daten.
        column_name (str): Name der Spalte, in der Outlier identifiziert werden sollen.

    Returns:
        pd.DataFrame: DataFrame mit den Outlier-Werten.
    """
    # Berechnung des IQR (Interquartilsabstand)
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.2 * IQR
    upper_bound = Q3 + 1.2 * IQR

    # Identifikation der Outlier
    outliers_iqr = df[(df[column_name] < lower_bound) | (df[column_name] > upper_bound)]

    # Berechnung des Z-Scores
    df['Z-Score'] = zscore(df[column_name])
    outliers_zscore = df[np.abs(df['Z-Score']) > 3]

    print(f"Anzahl der Outlier mit IQR-Methode: {len(outliers_iqr)}")
    print(f"Anzahl der Outlier mit Z-Score-Methode: {len(outliers_zscore)}")

    # Boxplot zur Visualisierung der Outlier
    plt.figure(figsize=(12, 5))
    sns.boxplot(x=df[column_name])
    plt.title(f"Boxplot der {column_name} mit Outliern")
    plt.show()

    # Histogramm der Verteilung
    plt.figure(figsize=(12, 5))
    sns.histplot(df[column_name], bins=30, kde=True, color="blue")
    plt.axvline(lower_bound, color="red", linestyle="--", label="IQR Untergrenze")
    plt.axvline(upper_bound, color="red", linestyle="--", label="IQR Obergrenze")
    plt.legend()
    plt.title(f"Histogramm der {column_name}")
    plt.show()

    # Rückgabe der Outlier
    return outliers_iqr

In [ ]:
# Erweiterung der Trainingsdaten
new_sample_count = 4540  # Anzahl der zu generierenden zusätzlichen Samples
extended_train_df = extend_regression_target_with_jitter(train_df, target_column, new_sample_count)

if test_size == 0.3:
        train_path = f"../Datasets/train_{target_column}_jitter_30_5000.pkl"
        test_path = f"../Datasets/test_{target_column}_30.pkl"
else:
        train_path = f"../Datasets/train_{target_column}_extended_jitter_5000.pkl"
        test_path = f"../Datasets/test_{target_column}_extended.pkl"
# Speichern der neuen Daten
extended_train_df.to_pickle(train_path)
test_df.to_pickle(test_path)

In [ ]:
# Testset jittern
test_df_jittered = extend_regression_target_with_jitter(test_df, target_column, 500)

In [ ]:
# Definiere die nominalen Werte
nominal_values = np.array([38, 40, 42])

# Funktion, um den nächstgelegenen nominalen Wert zu bestimmen
def assign_nominal_value(ist_value, nominal_values=nominal_values):
    distances = np.abs(nominal_values - ist_value)
    return nominal_values[np.argmin(distances)]

# Hilfssplte 
test_df_jittered['NominalProbenhoehe'] = test_df_jittered['Probenhoehe'].apply(assign_nominal_value)


### Visualisierung

In [ ]:
outlier_df_test_jittered = detect_and_visualize_outliers_by_group(test_df_jittered, "Probenhoehe",group_col="NominalProbenhoehe")

In [ ]:
# Entfernen der Outlier, falls erforderlich
clean_test_df = remove_outliers_by_group(test_df_jittered, column_name="Probenhoehe", group_col="NominalProbenhoehe")

In [ ]:
#Speichern des Testsets
clean_test_df.to_pickle(f"../Datasets/test_{target_column}_jittered.pkl")

In [ ]:
# Anwendung auf die Spalte "Probenhoehe"
print("Erweiterte Daten:")
outlier_df = detect_and_visualize_outliers(extended_train_df, "Probenhoehe")

# Die Outlier anzeigen
print(outlier_df)

# Erweiterung Bauteil Temperatur

### Hilfsfunktionen

In [ ]:
def detect_and_visualize_outliers_by_group(df, column_name, group_col):
    """
    Identifiziert und visualisiert Outlier in 'column_name' getrennt nach 'group_col'.
    
    Args:
        df (pd.DataFrame): DataFrame mit den Daten.
        column_name (str): Name der Spalte, in der Outlier identifiziert werden sollen.
        group_col (str): Name der Spalte, nach der gruppiert werden soll (z.B. 'NominalProbenhoehe').

    Returns:
        pd.DataFrame: DataFrame mit allen IQR-Outliern aus allen Gruppen.
        pd.DataFrame: DataFrame mit allen Z-Score-Outliern aus allen Gruppen.
    """
    # Listen zum Sammeln aller Outlier pro Gruppe
    outliers_iqr_list = []
    outliers_zscore_list = []

    # Gruppieren nach der gewünschten Spalte (z.B. nominale Probenhöhe)
    grouped = df.groupby(group_col)

    for group_value, group_df in grouped:
        print(f"\n--- Auswertung für {group_col} = {group_value} ---")

        # 1) IQR-Methode
        Q1 = group_df[column_name].quantile(0.25)
        Q3 = group_df[column_name].quantile(0.75)
        IQR = Q3 - Q1
        
        # Grenzen anpassen, falls gewünscht (hier 1.2*IQR wie in deinem Code)
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers_iqr = group_df[
            (group_df[column_name] < lower_bound) | (group_df[column_name] > upper_bound)
        ]
        outliers_iqr_list.append(outliers_iqr)

        print(f"  Anzahl Outlier (IQR) für Gruppe {group_value}: {len(outliers_iqr)}")

        # 2) Z-Score-Methode
        # Wir berechnen den Z-Score nur auf die Teilgruppe
        group_df = group_df.copy()  # damit wir den Original-DataFrame nicht überschreiben
        group_df['Z-Score'] = zscore(group_df[column_name])
        outliers_z = group_df[np.abs(group_df['Z-Score']) > 3]
        outliers_zscore_list.append(outliers_z)
        
        print(f"  Anzahl Outlier (Z-Score) für Gruppe {group_value}: {len(outliers_z)}")

        # -- Visualisierung pro Gruppe (optional) --
        # Boxplot
        plt.figure(figsize=(10, 4))
        sns.boxplot(x=group_df[column_name])
        plt.title(f"Boxplot der {column_name} (Gruppe: {group_value})")
        plt.show()

        # Histogramm
        plt.figure(figsize=(10, 4))
        sns.histplot(group_df[column_name], bins=30, kde=True, color="blue")
        plt.axvline(lower_bound, color="red", linestyle="--", label="IQR Untergrenze")
        plt.axvline(upper_bound, color="red", linestyle="--", label="IQR Obergrenze")
        plt.legend()
        plt.title(f"Histogramm der {column_name} (Gruppe: {group_value})")
        plt.show()

    # Zusammenführen aller Outlier aus allen Gruppen
    outliers_iqr_all = pd.concat(outliers_iqr_list) if outliers_iqr_list else pd.DataFrame()
    outliers_zscore_all = pd.concat(outliers_zscore_list) if outliers_zscore_list else pd.DataFrame()

    return outliers_iqr_all, outliers_zscore_all

def remove_outliers_by_group(df, column_name, group_col):
    """
    Entfernt Outlier aus dem DataFrame, gruppiert nach einer nominalen Spalte.
    Die Outlier werden mithilfe der IQR-Methode innerhalb jeder Gruppe identifiziert.
    Anschließend wird der DataFrame bereinigt und die Gruppierungsspalte entfernt.

    Args:
        df (pd.DataFrame): Ursprünglicher DataFrame.
        column_name (str): Name der Spalte, in der Outlier erkannt werden sollen.
        group_col (str): Name der Spalte, die die nominalen Werte (z.B. 38, 40, 42) enthält.

    Returns:
        pd.DataFrame: Bereinigter DataFrame ohne Outlier und ohne die Gruppierungsspalte.
    """
    # Kopie erstellen, um das Original beizubehalten
    df_clean = df.copy()
    outlier_indices = []

    # Gruppierung nach der nominalen Probenhöhe (die zuvor aus den IST-Werten erstellt wurde)
    for group_value, group_df in df_clean.groupby(group_col):
        Q1 = group_df[column_name].quantile(0.25)
        Q3 = group_df[column_name].quantile(0.75)
        IQR = Q3 - Q1
        
        # Grenzen der IQR-Methode (hier 1.2 * IQR, kann angepasst werden)
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Identifikation der Outlier in dieser Gruppe
        group_outliers = group_df[(group_df[column_name] < lower_bound) | (group_df[column_name] > upper_bound)]
        outlier_indices.extend(group_outliers.index.tolist())

    # Doppelte Indizes entfernen
    outlier_indices = list(set(outlier_indices))
    
    # Entferne die Zeilen mit Outliern aus dem DataFrame
    df_clean = df_clean.drop(index=outlier_indices)
    
    # Entferne die nominale Spalte, die zur Gruppierung genutzt wurde
    df_clean = df_clean.drop(columns=[group_col])
    
    return df_clean


def apply_smogn_with_inverse_scaling_and_fallback(
    df, target_col, mode_ranges, samples_per_mode=500, k=5,
    rel_thres=0.8, random_state=42, min_k=3
):
    df = df.copy()
    np.random.seed(random_state)

    # --- Kategorische Spalten erkennen und label-encoden ---
    cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
    label_encoders = {}
    for col in cat_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    # --- Fehlende Werte behandeln ---
    imputer = SimpleImputer(strategy='median')
    df[df.columns] = imputer.fit_transform(df)

    # --- Skalierung ---
    scaler = StandardScaler()
    X = df.drop(columns=[target_col])
    y = df[target_col]
    X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
    df_scaled = pd.concat([X_scaled, y.reset_index(drop=True)], axis=1)

    all_augmented = []

    for min_val, max_val in mode_ranges:
        subset = df_scaled[(df_scaled[target_col] >= min_val) & (df_scaled[target_col] <= max_val)].copy()
        print(f"[DEBUG] Bereich {min_val}-{max_val}: subset.shape = {subset.shape}, k = {k}")

        if subset.shape[0] < min_k + 1:
            print(f"[WARN] Zu wenige Daten in Bereich {min_val}-{max_val}, überspringe...")
            continue

        subset.reset_index(drop=True, inplace=True)

        rel_method = "manual"
        rel_ctrl_pts_rg = [
            [min_val - 20, 0.0],
            [min_val + (max_val - min_val) / 2, 1.0],
            [max_val + 20, 0.0]
        ]

        success = False
        current_k = k
        while not success and current_k >= min_k:
            try:
                smogn_result = smogn.smoter(
                    data=subset.copy(),
                    y=target_col,
                    samp_method="balance",
                    rel_method=rel_method,
                    rel_ctrl_pts_rg=rel_ctrl_pts_rg,
                    rel_thres=rel_thres,
                    k=current_k
                )
                new_rows = smogn_result.iloc[subset.shape[0]:].copy()
                if new_rows.empty:
                    raise ValueError("Keine neuen Daten erzeugt durch SMOGN.")
                if new_rows.shape[0] > samples_per_mode:
                    new_rows = new_rows.sample(n=samples_per_mode, random_state=random_state)
                new_rows["synthetisch"] = 1
                all_augmented.append(new_rows)
                success = True
            except Exception as e:
                print(f"[FALLBACK] SMOGN fehlgeschlagen für Bereich {min_val}-{max_val}. Fehler: {e}")
                current_k -= 1

        if not success:
            try:
                jittered = subset.sample(n=samples_per_mode, replace=True, random_state=random_state)
                jitter_cols = jittered.select_dtypes(include=np.number).columns.drop(target_col)
                jittered[jitter_cols] += np.random.normal(0, 0.01, size=jittered[jitter_cols].shape)
                jittered["synthetisch"] = 1
                all_augmented.append(jittered)
                print(f"[FALLBACK] Jitter erfolgreich für Bereich {min_val}-{max_val}")
            except Exception as e:
                print(f"[ERROR] Auch Jitter fehlgeschlagen für Bereich {min_val}-{max_val}: {e}")
                continue

    if not all_augmented:
        raise ValueError("Keine synthetischen Daten erzeugt.")

    df_augmented_scaled = pd.concat(all_augmented, ignore_index=True)
    X_new = df_augmented_scaled.drop(columns=[target_col, "synthetisch"])
    X_new_inverse = pd.DataFrame(scaler.inverse_transform(X_new), columns=X.columns)
    df_augmented = pd.concat([
        X_new_inverse,
        df_augmented_scaled[[target_col, "synthetisch"]].reset_index(drop=True)
    ], axis=1)

    for col in cat_cols:
        le = label_encoders[col]
        df_augmented[col] = df_augmented[col].round().astype(int).clip(0, len(le.classes_) - 1)
        df_augmented[col] = le.inverse_transform(df_augmented[col])

    df_original = df.copy()
    df_original["synthetisch"] = 0

    df_final = pd.concat([df_original, df_augmented], ignore_index=True)
    return df_final


In [ ]:
# **Spezifische Anwendung für Bauteil Temp**
target_column = "Bauteil_Temp"

# Datensatz laden
df = pd.read_pickle("../datasets/new_features.pkl")
# Definiere die nominalen Werte

nominal_values = np.array([800,1200]) # Ungefähre Temperauren in original Daten

# Funktion, um den nächstgelegenen nominalen Wert zu bestimmen
def assign_nominal_value(ist_value, nominal_values=nominal_values):
    distances = np.abs(nominal_values - ist_value)
    return nominal_values[np.argmin(distances)]

# Hilfssplte 
df['NominalBauteil_Temp'] = df['Bauteil_Temp'].apply(assign_nominal_value)
print("Originale Daten:")
outlier_df1 = detect_and_visualize_outliers_by_group(df, "Bauteil_Temp",group_col="NominalBauteil_Temp")


In [ ]:
# Entfernen von Outliern 
df_clean = remove_outliers_by_group(df, column_name="Bauteil_Temp", group_col="NominalBauteil_Temp")

# Split in Training und Test
train_df, test_df = train_test_split(df_clean, test_size=test_size, random_state=42)
train_df.to_pickle(f"../datasets/train_{target_column}_clean.pkl")
test_df.to_pickle(f"../datasets/test_{target_column}_clean.pkl")
irrelevant_columns = ['Material_con', 'Position_con', 'Ergebnis_con', 'richtig_verbaut', 'Zeit', 'VersuchID','umformzeit','Probenhoehe',
                      ]
df_reduced = df_clean.drop(columns=irrelevant_columns, errors='ignore')


## SMOGN

In [ ]:
mode_ranges = [(730, 780),(780,830),(830,880),(1080,1180) ,(1180, 1250)]
df_augmented = apply_smogn_with_inverse_scaling_and_fallback(
    df_reduced,
    target_col="Bauteil_Temp",
    mode_ranges=mode_ranges,
    samples_per_mode=1000  # optional mehr Samples pro Bereich
)

In [ ]:
# Hilfssplte 
df_augmented['NominalBauteil_Temp'] = df_augmented['Bauteil_Temp'].apply(assign_nominal_value)

outlier_df_test_jittered = detect_and_visualize_outliers_by_group(df_augmented, "Bauteil_Temp",group_col="NominalBauteil_Temp")
clean_train_df = remove_outliers_by_group(df_augmented, column_name="Bauteil_Temp", group_col="NominalBauteil_Temp")

In [ ]:
# Hilfssplte 
clean_train_df['NominalBauteil_Temp'] = clean_train_df['Bauteil_Temp'].apply(assign_nominal_value)

outlier_df_test_jittered = detect_and_visualize_outliers_by_group(clean_train_df, "Bauteil_Temp",group_col="NominalBauteil_Temp")

# Entfernen NominalBauteil_Temp
clean_train_df = clean_train_df.drop(columns=['NominalBauteil_Temp',], errors='ignore')

# Wieder gruppieren
clean_train_df['Bauteil_Temp_Gruppe'] = clean_train_df['Bauteil_Temp'].round()

In [ ]:
clean_train_df

In [ ]:
# Annahme: Die Features sind alle Spalten außer der Zielvariable & kategorischen Spalten
features = [col for col in clean_train_df.columns if col not in ['Ergebnis_con', 'VersuchID', 'Material_con', 'Position_con', 'richtig_verbaut','Probenhoehe']]


# Boxplots getrennt nach Bauteil_Temp
# for feature in features:
#     plt.figure(figsize=(10, 6))
#     sns.boxplot(x='synthetisch', y=feature, data=clean_train_df, palette="Set2")
    
#     plt.title(f'Vergleich der Verteilung für {feature} nach Klassen')
#     plt.xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
#     plt.ylabel(feature)
#     plt.legend(title="Bauteil_Temp", loc="upper right")
    
#     plt.show()

# Setze die Features für die Subplots
features_to_plot = ["auftreffposition", "Verkippung_2_Min"]

# Erstelle die Subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))  # 1 Zeile, 2 Spalten

for i, feature in enumerate(features_to_plot):
    sns.boxplot(x='synthetisch', y=feature, data=clean_train_df, palette="Set2", ax=axes[i])
    axes[i].set_title(f'Vergleich der Verteilung für {feature} nach Klassen')
    axes[i].set_xlabel("Synthetisch (True = künstliche Datenpunkte, False = reale Datenpunkte)")
    axes[i].set_ylabel(feature)

# Subplots platzsparend anordnen
plt.tight_layout()
plt.show()

In [ ]:
# Visualisierung Verteilung der Daten: original vs synthetisch

fig, axs = plt.subplots(1, 2, figsize=(15, 5))

sns.histplot(df_clean['Bauteil_Temp'], bins=300, kde=True, color="#4C72B0", ax=axs[0])
axs[0].set_title("Verteilung der Bauteiltemperatur im bereinigten originalen Datensatz")
axs[0].set_xlabel("Bauteiltemperatur (°C)")
axs[0].set_ylabel("Häufigkeit")
axs[0].grid(True)

sns.histplot(df_augmented['Bauteil_Temp'], bins=300, kde=True, color="#4C72B0", ax=axs[1])
axs[1].set_title("Verteilung der Bauteiltemperatur im augmentierten Datensatz")
axs[1].set_xlabel("Bauteiltemperatur (°C)")
axs[1].set_ylabel("Häufigkeit")
axs[1].grid(True)

plt.tight_layout()
plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.savefig("BauteilTempVerteilungSMOGN.svg", format="svg", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
# Ausreißer entfernen 
df_train = clean_train_df.drop(columns=['synthetisch','Bauteil_Temp_Gruppe'], errors='ignore')
#df_train.to_pickle(f"../datasets/train_{target_column}_erweitert.pkl")

In [ ]:
df['Bauteil_Temp']

In [ ]:
# **Spezifische Anwendung für Bauteil Temp**
target_column = "Bauteil_Temp"

# Datensatz laden
df = pd.read_pickle("../datasets/new_features.pkl")

# Split in Training und Test
train_df, test_df = train_test_split(df, test_size=test_size, random_state=42)

# Erweiterung der Trainingsdaten
extended_train_df = extend_regression_target_with_jitter(train_df, target_column, new_sample_count)

if test_size == 0.3:
        train_path = f"../Datasets/train_{target_column}_smote_30.pkl"
        test_path = f"../Datasets/test_{target_column}_30.pkl"
else:
        train_path = f"../Datasets/train_{target_column}_smote.pkl"
        test_path = f"../Datasets/test_{target_column}.pkl"


In [ ]:
# Speichern der neuen Daten
# extended_train_df.to_pickle(train_path)
# test_df.to_pickle(test_path)